<a href="https://colab.research.google.com/github/TomasMendozaHN/ICDF_Class/blob/main/05182022_FasterRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>